In [6]:
#! pip install ultralytics 
#! pip install psycopg2
import numpy as np
#import cv2
#import matplotlib.pyplot as plt
import os
import psycopg2
#import im_utils
import db_ops
import pandas as pd

Connecting to the PostgreSQL database...
Connection successful


#### Get the list of test labels

In [14]:
test_lbls = ['5-hour energy', '99', 'AW', 'Acadia', 'Arizona Tea', 'Bang', 'Big Gulp', 'Bill Miller', 'Boulevard Brewing', 'Brisk', 'Bud', 'Budweiser', 'Burger King', 'Busch', 'Camel', 'Cheetos', 'Coca Cola', 'Coors', 'Corona', 'Dasani', 'Dasy', 'David', 'Deer Park', 'Dorritos', 'Dunkin Donuts', 'Evan Williams', 'Fanta', 'Fireball', 'Flow', 'Fruit Punch', 'Gatorade', 'GoGo', 'Google', 'Goya', 'Grace', 'Green Mountain', 'Heineken', 'Hersheys', 'Hill Country Fair', 'Icehouse Edge', 'Jarritos', 'Jim Beam', 'Kirkland', 'Kit Kat', 'Kool-Aid', 'La Yogurt', 'LaCroix', 'Lays', 'Lybbis', 'Marlboro', 'McDonalds', 'Mentos', 'Michelob', 'Miller', 'Milwakee Best Ice', 'MnM', 'Modelo', 'Monster', 'Mountain Dew', 'Natural Ice', 'New Amsterdam', 'Niagara', 'Oreo', 'Pabst Blue Ribbon', 'Pacifico', 'Pampers', 'Parliament', 'Peaceful Fruits', 'Perrier', 'Poland Spring', 'Popeyes', 'PurAqua', 'Redbull', 'Ross Ouzo', 'Sams Club', 'Samuel Adams', 'Sangria Senorial', 'Sanpple', 'Seven eleven', 'Shasta', 'Silver Falls', 'Slurpee', 'Smirnoff', 'Snickers', 'Sprite', 'Stella Artois', 'Turkey Hill', 'Twix', 'Two Ball', 'Vitamin', 'Welchs', 'Yoohoo']

In [100]:
label_path = './test_dataset/train/labels'
files = os.listdir(label_path)
file_names = []
labels = []
for file in files:
    file_labels = ''
    with open(label_path + '/'+file, 'r') as label_file:
        for line in label_file:
            idx = int(line.split(' ')[0])
            file_labels = file_labels + test_lbls[idx] + ','
        labels.append(file_labels.strip())
        file_names.append(file.split('_jpg')[0])
        
test_labels_df = pd.DataFrame({'file_name':file_names,'labels': labels})
test_labels_df = test_labels_df.set_index('file_name')
test_labels_df

#### Get the full list of avavliable images

In [104]:
query = f'''
SELECT bx.box_id as box_id
	,im.s3_uri AS uri
	, br.brand_name as brand
FROM box AS bx 
	LEFT JOIN image AS im
		ON im.image_id = bx.image_id
	LEFT JOIN brand as br
		ON br.brand_id = bx.brand_id 
ORDER BY bx.box_id	;
'''
boxes_df = db_ops.select_to_pandas(query, True, True)

c:\Users\Artem\Documents\Source_Code\litter-imagery-brand-recognition\db_ops.py:57: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


In [143]:
boxes_df['file_name'] = boxes_df['uri'].apply(lambda x: x.split('/')[-1].split('.')[0])
processed_images = boxes_df.groupby('file_name')['brand'].apply(list).reset_index(name='predicted_brands')
processed_images = processed_images.set_index('file_name')
processed_images['fouind_case'] = processed_images['predicted_brands'].apply(lambda x: len(x))
processed_images.head(3)

,predicted_brands,fouind_case
file_name,,
00BbrOV1vjd2Me36yX4jqMNzBWqQQ30ZLcYW3Kve,[zz_unbranded],1
00c8QBSWDnnUPMkYr7emb2veo5P2IfrjsdgJ8Ya6,"[corona, modelo, zz_unbranded, modelo, modelo,...",7
01euAki2lAbE4krE9NM35c48vu3ZThU1DbIp0F1o,"[heineken, zz_unbranded, heineken]",3


In [144]:
result = test_labels_df.join(processed_images)
result['labels'] = result['labels'].apply(lambda x: x.lower())
result['predicted_brands'] = result['predicted_brands'].apply(lambda x: str(x).replace('[', ' '))
result['predicted_brands'] = result['predicted_brands'].apply(lambda x: str(x).replace(']', ' '))
#result['predicted_brands'] = result['predicted_brands'].apply(lambda x: str(x).replace('zz_unbranded', 'nan'))
#result['predicted_brands'] = result['predicted_brands'].apply(lambda x: str(x).replace("'", '').strip())
result['labels'] = result['labels'].apply(lambda x: 'nan' if x == '' else x)

In [146]:
result['correct'] = 0
result['cases'] = 1

In [152]:
for i in range(len(result)):
    result['cases'].iloc[i] = len(result['labels'].iloc[i][:-2].split(','))
    if result['labels'].iloc[i] == result['predicted_brands'].iloc[i]:
        result['correct'].iloc[i] = len(result['labels'].iloc[i].split(','))
        result['cases'].iloc[i] = result['correct'].iloc[i]
result['fouind_case'].fillna(1, inplace=True)
#result.to_csv('resuts.csv', sep = '\t')
result

C:\Users\Artem\AppData\Local\Temp\ipykernel_22636\681215665.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  result['cases'].iloc[i] = len(result['labels'].iloc[i][:-2].split(','))
C:\Users\Artem\AppData\Local\Temp\ipykernel_22636\6812156

,labels,predicted_brands,fouind_case,correct,cases
file_name,,,,,
0orY3PkxVw6mPVeft3clwqWJrhiDlY03afoz33ZA,nan,nan,1.0,1,1
1i1eXWTp6MrJwU05wKgvAr2XT3p35LPPLKJDZnNM,nan,'zz_unbranded',1.0,0,1
2R7Sp85dVjjkaxudOXW79jnZqqDKkFtx4vnaNhTC,"corona,corona,",nan,1.0,0,2
3E3y1cX5WaUTToWvBLkdJn0f8KZEyvxenfdsgyoD,nan,nan,1.0,1,1
3IIxk6EMH9n7Un7xYwsPtTCwBPJM4e16TkiehifB,"cheetos,","'zz_unbranded', 'zz_unbranded', 'zz_unbranded'",3.0,0,1
...,...,...,...,...,...
z8aXJgaWPLXxG4fz573nVPly1vH8yCtfqXcAefog,nan,'zz_unbranded',1.0,0,1
ZlbBeVYQJFmARnOoP4xVvuYnCpqKkZQGJoKOBVR9,"modelo,","'modelo', 'modelo'",2.0,0,1
Zw1CDxqtdI7uq9jZTmzIpjKhN8ayXv2EPfA6jdk6,"modelo,modelo,modelo,modelo,modelo,modelo,mode...","'modelo', 'modelo', 'modelo'",3.0,0,13


In [153]:
result['fouind_case'].sum()

424.0